install python packages and popper (for ocd)



In [ ]:
!sudo apt-get install poppler-utils -y
!pip3 install -q pdf2image requests numpy tqdm
!pip install -q git+https://github.com/illuin-tech/colpali
!pip install -q uv
!uv pip install --system qdrant-client stamina rich

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Using Python 3.10.12 environment at /usr
Audited 3 packages in 197ms


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from io import BytesIO
from qdrant_client import QdrantClient
from qdrant_client.http import models
from colpali_engine.models import ColQwen2, ColQwen2Processor
from pdf2image import convert_from_path
from IPython.display import display
from io import BytesIO
import base64
from PIL import Image
import torch
import cv2
import numpy as np
from pathlib import Path
from typing import List, Dict
from tqdm import tqdm
import json
import os

def encode_base64_image(image):
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    return str(base64.b64encode(buffered.getvalue()), "utf-8")


def decode_base64_image(base64_string):
    # Decode the base64 string into bytes
    image_data = base64.b64decode(base64_string)

    # Convert the bytes back into an image
    buffered = BytesIO(image_data)
    image = Image.open(buffered)

    return image

def unique_positive_hash_64bit(value):
    hash_value = hash(value)
    unsigned_hash = hash_value & ((1 << 64) - 1)  # Treat as 64-bit unsigned
    return unsigned_hash


def resize_image(image, max_height=800):
    width, height = image.size
    if height > max_height:
        ratio = max_height / height
        new_width = int(width * ratio)
        new_height = int(height * ratio)
        return image.resize((new_width, new_height))
    return image


def save_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=2)

def load_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)

def combine_json_files(directory: str):
    combined_data = []

    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)
            data = load_json(file_path)
            combined_data.extend(data)

    return combined_data

### Load the model

We use device map auto to load the model on the available GPU if available, otherwise on the CPU or MPS if available.


In [ ]:
model = ColQwen2.from_pretrained(
        "vidore/colqwen2-v1.0",
        torch_dtype=torch.bfloat16,
        device_map="cuda",  # or "mps" if on Apple Silicon
    ).eval()
processor = ColQwen2Processor.from_pretrained("vidore/colqwen2-v1.0")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [ ]:
data = load_json("/content/drive/MyDrive/Data/manuals-data.json")

In [ ]:
pdf_titles = list({"title": item} for item in list(set(item["title"] for item in data)))

for pdf in pdf_titles:
    title = pdf["title"]
    print(title)
    pdf_pages = []
    images = convert_from_path(f"/content/drive/MyDrive/MiR200/{title}.pdf")
    for i, image in tqdm(enumerate(images), total=len(images)):
        processed_image = processor.process_images([image])
        with torch.no_grad():
            processed_image = {k: v.to(model.device) for k, v in processed_image.items()}
            embedding = model(**processed_image)
            embs = embedding.to("cpu").squeeze(0)
        pdf_pages.append({
            "title": title,
            "image_embeddings": embs.float().numpy().tolist(),
            "page_number": i+1
        })
    save_json(pdf_pages, "/content/drive/MyDrive/Embeddings/"+title+".json")



Troubleshoot docking issues 1.2_en


100%|██████████| 33/33 [00:15<00:00,  2.09it/s]


Troubleshoot Wifi and Network issues with MiR robots and MiR fleet


100%|██████████| 22/22 [00:10<00:00,  2.10it/s]


MiR 24V Battery Troubleshooting and Technical Guide 2.1_en


...

100%|██████████| 187/187 [01:28<00:00,  2.11it/s]


Einbau der 24-V-Batterie mit erhöhter Kapazität am MiR100 und MiR200 1.0_de


100%|██████████| 54/54 [00:25<00:00,  2.11it/s]


MiR200 Quick Start 2.3_en


100%|██████████| 40/40 [00:18<00:00,  2.12it/s]


In [ ]:
text_embeddings = load_json("/content/drive/MyDrive/Data/chunked-and-embedded-manuals.json")

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('Alibaba-NLP/gte-multilingual-base')
# transform sparse and dense vectors to right form
for chunk in text_embeddings:
    result = {
        "values": [],
        "indices":[]
    }
    for idx, val in chunk["sparse_embeddings"][0].items():
        result["values"].append(val)
        result["indices"].append(idx)
    chunk["sparse_embeddings"] = result
    chunk["dense_embeddings"] = chunk["dense_embeddings"][0]

In [ ]:
save_json(text_embeddings, "/content/drive/MyDrive/Data/chunked-and-embedded-manuals.json")

In [ ]:
qdrant_documents = []
idx = 1
for chunk in text_embeddings:
    # indices have to be integer
    qdrant_documents.append(
            models.PointStruct(
                id= idx,
                vector={
                    "dense": chunk["dense_embeddings"],
                    "sparse": chunk["sparse_embeddings"]
                },
                payload={
                    "title": chunk["title"],
                    "page_number": chunk["page_number"],
                    "chunk_number": chunk["chunk_number"],
                    "text": chunk["text"],
                    "headings": chunk["headings"],
                    "language": chunk["language"],
                    "label": chunk["label"],
                    "image": chunk["base64"],

                },
            )
        )
    idx += 1

In [ ]:
len(text_embeddings[0]["dense_embeddings"])

768

In [ ]:
collection_name = "chunked-and-embedded-manuals"
QDRANT_API_KEY = ""
qdrant_client = QdrantClient(
     url="https://c1c6b76d-3e82-4265-8850-542922eeb4b9.us-east4-0.gcp.cloud.qdrant.io:6333/dashboard#/collections",
     api_key=QDRANT_API_KEY,
 )

qdrant_client.recreate_collection(
    collection_name=collection_name,
    on_disk_payload=True,
    vectors_config = {
        "dense": models.VectorParams(
        size=768,
        distance=models.Distance.DOT,
    )},
    sparse_vectors_config = {
        "sparse": models.SparseVectorParams(
            index=models.SparseIndexParams(
                on_disk=False,
            )
        )
    }
)

<ipython-input-8-5709f231fa47>:8: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


True

In [ ]:
import stamina


@stamina.retry(on=Exception, attempts=3)
def upsert_to_qdrant(batch):
    try:
        qdrant_client.upsert(
            collection_name=collection_name,
            points=batch,
            wait=False,
        )
    except Exception as e:
        print(f"Error during upsert: {e}")
        return False
    return True

In [ ]:
batch_size = 2  # Adjust based on your GPU memory constraints

# Use tqdm to create a progress bar
with tqdm(total=len(qdrant_documents), desc="Upserting Progress") as pbar:
    for i in range(0, len(qdrant_documents), batch_size):
        points = qdrant_documents[i : i + batch_size]

        # Upload points to Qdrant
        try:
            upsert_to_qdrant(points)
        # clown level error handling here 🤡
        except Exception as e:
            print(f"Error during upsert: {e}")
            continue

        # Update the progress bar
        pbar.update(batch_size)

print("Upserting complete!")

Upserting Progress: 3530it [03:07, 18.86it/s]

Upserting complete!


# Per page one entry

In [ ]:
data = combine_json_files("/content/drive/MyDrive/Embeddings")
metadata = load_json("/content/drive/MyDrive/Data/embedded-manuals.json")

In [ ]:
lookup_dict = {
        (item['title'], item['page_number']): item['image_embeddings']
        for item in data
    }

for item in metadata:
    key = (item['title'], item['page_number'])
    if key in lookup_dict:
        item['image_embeddings'] = lookup_dict[key]

In [ ]:
for page in metadata:
    page["dense_embeddings"] = page["dense_embeddings"][0]
    result = {
        "values": [],
        "indices":[]
    }
    for idx, val in page["sparse_embeddings"][0].items():
        result["values"].append(val)
        result["indices"].append(int(idx))
    page["sparse_embeddings"] = result

In [ ]:
len(metadata[0]["dense_embeddings"])

768

In [ ]:
collection_name = "embedded-manuals"
QDRANT_API_KEY = ""
qdrant_client = QdrantClient(
    url="https://568e9069-f3eb-4ee2-8e22-52e7bad7d3b7.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key=QDRANT_API_KEY,
)

qdrant_client.recreate_collection(
    collection_name=collection_name,
    on_disk_payload=True,
    vectors_config = {
        "dense": models.VectorParams(
        size=768,
        distance=models.Distance.DOT,
    ),
        "multi": models.VectorParams(
        size=128,
        distance=models.Distance.COSINE,
        multivector_config=models.MultiVectorConfig(
            comparator=models.MultiVectorComparator.MAX_SIM
        ),
    ),},
    sparse_vectors_config = {
        "sparse": models.SparseVectorParams(
            index=models.SparseIndexParams(
                on_disk=False,
            )
        )
    }
)

<ipython-input-35-8f1e8c0184a3>:8: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


True

In [ ]:
import numpy as np

qdrant_documents_multi = []
id = 1
for chunk in metadata:
      qdrant_documents_multi.append(
            models.PointStruct(
                id= id,
                vector={
                  "dense": chunk["dense_embeddings"],
                    "sparse": chunk["sparse_embeddings"],
                    "multi": chunk["image_embeddings"]
                },
                payload={
                    "title": chunk["title"],
                    "page_number": chunk["page_number"],
                    "text": chunk["text"],
                    "headings": chunk["headings"],
                    "language": chunk["language"],
                    "label": chunk["label"],
                    "image": chunk["base64"],

                },
            )
        )
      id += 1

In [ ]:
import stamina


@stamina.retry(on=Exception, attempts=3)
def upsert_to_qdrant(batch):
    try:
        qdrant_client.upsert(
            collection_name=collection_name,
            points=batch,
            wait=False,
        )
    except Exception as e:
        print(f"Error during upsert: {e}")
        return False
    return True

In [ ]:
batch_size = 2  # Adjust based on your GPU memory constraints

# Use tqdm to create a progress bar
with tqdm(total=len(qdrant_documents_multi), desc="Upserting Progress") as pbar:
    for i in range(0, len(qdrant_documents_multi), batch_size):
        points = qdrant_documents_multi[i : i + batch_size]

        # Upload points to Qdrant
        try:
            upsert_to_qdrant(points)
        # clown level error handling here 🤡
        except Exception as e:
            print(f"Error during upsert: {e}")
            continue

        # Update the progress bar
        pbar.update(batch_size)

print("Upserting complete!")

Upserting Progress: 100%|██████████| 1078/1078 [12:14<00:00,  1.47it/s]

Upserting complete!


In [ ]:
ids = []
idx = 1
for page in text_embeddings:
    id = dict()
    id["title"] = page["title"]
    id["page_number"] = page["page_number"]
    id["id"] = idx
    ids.append(id)
    idx += 1


In [ ]:
save_json(text_embeddings, "/content/drive/MyDrive/Data/chunked-and-embedded-manuals.json")

In [ ]:
def is_uint64(number):
    return isinstance(number, int) and 0 <= number < 2**64

val = unique_positive_hash_64bit(page["title"]+str(page["page_number"]))

is_uint64(val)

True